## Notebook Redis

- Riezu, German

This notebook is based on the Redis tutorial that you can find on: https://realpython.com/python-redis/

<b>You will need to set up a Redis server. You can clone the Redis github and execute the redis-server file. You will have to change the server password to "test" so that it works directly. If you don't want to change it you can also modify the Redis object down below deleting the "password = 'test" argument.</b>

D'abord on va tester comment faire pour communiquer REDIS avec un client qui n'est pas controlé par la ligne de commandes et, par example, écrit en Python en utilisant des APIS. Il faut que le serveur soit configuré avec un mot de pas = "test"

In [ ]:
!pip install redis

In [2]:
import redis   #importer la bibliotheque
#On crée un client redis sur la machine locale, sur la porte 6379(default) et qui utilise "test" comme mot de pas
r = redis.Redis(host='localhost',
                port=6379, 
                password = 'test')

#Pour tester on va introduire dans REDIS deux élements clé-valeur et on va à demander la valeur de la clé Bahamas
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})
r.get("Bahamas").decode("utf-8")

'Nassau'

REDIS est très utilisé car il est capable de faire beaucoup de transactions SET/GET très rapidement et donc on va le tester.

In [3]:
a=1
b=2
c=3
d=4
e=5

In [11]:
%%timeit
r.set(a,a)
r.set(b,b)
r.set(c,c)
r.set(d,d)
r.set(e,e)

554 µs ± 20.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
dictTest ={a:a,b:b,c:c,d:d,e:e}

In [14]:
%%timeit
r.mset(dictTest)

127 µs ± 567 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [2]:
d = {i:i for i in range(10000)}

In [3]:
%%timeit #avec le pipelining on pourrait le faire plus rapidement
r.mset(d)

64.9 ms ± 5.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
r.flushall()  #On éfface toute le data qu'on avait sur tous(15) les databases REDIS

True

On va créer trois élements hash qui corréspondent à trois modèles de chapeaux différents. Chaque structure a les mêmes champs qui décrivent l'état de chaque chapeau.  

In [18]:
import random

random.seed(444)
hats = {f"hat:{random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

Pipelining est une méthode qui permet d'envoyer plusieurs commands au même temps pour reduire le temps totale. Dans la cell suivante, on va mettre le dictionaire que l'on a mis dans la dernière cell.

In [20]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        pipe.hmset(h_id, hat)
    pipe.execute()

In [7]:
r.keys()

[b'hat:56854717', b'hat:1236154736', b'hat:1326692461']

In [8]:
import logging

logging.basicConfig()

class OutOfStockError(Exception):

def purchaseItem(client, itemid: int,itemsBought) -> None:
    with client.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                # Voir ce qu'on a en stock               
                pipe.watch(itemid)
                nleft: bytes = client.hget(itemid, "quantity") 
                if int(nleft.decode("utf-8")) >= itemsBought:  #Comparation items qui restent et items achetés
                    pipe.multi()
                    pipe.hincrby(itemid, "quantity", -itemsBought)  #mise à jour
                    pipe.hincrby(itemid, "npurchased", itemsBought)
                    pipe.execute()
                    break
                else:
                   
                    pipe.unwatch()
                    raise OutOfStockError(
                        f"Sorry, {itemid} is out of stock!"
                    )
            except redis.WatchError:
                #Log le nombre d'erreurs
                error_count += 1
                logging.warning(
                    "WatchError #%d: %s; retrying",
                    error_count, itemid
                )
    return client.hgetall(itemid)

A nous de jouer et voir comment ça réagit:

In [21]:
purchaseItem(r,b'hat:1326692461',1)

{b'color': b'black',
 b'price': b'49.99',
 b'style': b'fitted',
 b'quantity': b'999',
 b'npurchased': b'1'}